# Stock Price Prediction 
Here we will focus on analyzing the prices of S&P500, Dow Jones Industrial Average and Gold over the period of 20 years from 01/01/2001 - 12/31/2020. We have built a time series predictive model below. 

In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import iplot
import plotly.graph_objs as go
import yfinance as yf
import statsmodels.api as sm
from datetime import timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import coint
import warnings


warnings.filterwarnings("ignore")
plt.style.use('seaborn')

In [ ]:
data =[]
#Storing the data imported 
asset_ticker = ['GC=F','^GSPC','DJI']
#List of Ticker Symbols
asset_names = ['Gold','S&P 500','Dow_Jones Industrial Average']
#List of Names 

for ticker in asset_ticker:
    ft = yf.Ticker(ticker)
    df = ft.history(start='2001-01-01',end='2020-12-30')['Open']
    data.append(df)
#Fetching the Stock prices from Yahoo finance 

price = pd.concat([df for df in data],axis=1)

price.columns = asset_names 
#We will use actual names instead of symbols for column names

price.tail(3)
#Printing the top three rows 

# EXPLORATORY DATA ANALYSIS

In [ ]:
#Handaling the Missing Values
price.info()

In [ ]:
for column in price.columns:
    null_dt = price[price[column].isnull()].index 
    # find out date where it is null
    idx = np.searchsorted(price.index, null_dt)
    for i in range(len(idx)-1):
        if math.isnan(price.iloc[idx[i]+1][column]): 
            # if value afterwards is NaN, then average will also be NaN
            price[column].loc[null_dt[i]] = price.iloc[idx[i]-1][column] 
        elif math.isnan(price.iloc[idx[i]-1][column]):
            price[column].loc[null_dt[i]] = price.iloc[idx[i]+1][column] 
            # if value before is NaN, then average will also be NaN
        else:
            avg = (price.iloc[idx[i]+1][column] + price.iloc[idx[i]-1][column])/2
            price[column].loc[null_dt[i]] = avg

In [ ]:
yearly_price_open = price.resample('1Y').first()
yearly_price_close = price.resample('1Y').last()
yearly_price_high = price.resample('1Y').max()
yearly_price_low = price.resample('1Y').min()

#Using Open High Low Close Chart for analysis
for column in price.columns:
    fig = (go.Figure(data=go.Ohlc(
                                    x=yearly_price_open[column].index.year,
                                    open=yearly_price_open[column],
                                    high=yearly_price_high[column],
                                    low=yearly_price_low[column],
                                    close=yearly_price_close[column])))
    fig.update_layout(title_text=column,
                  title={
                    'y':0.85,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},)
    fig.show()

In [ ]:
#Exploring yearly rates 
def percent(df): 
    # function for transforming float DataFrame to DataFrame with percentages
    for column in df.columns:
        df[column] = df[column].apply(lambda x: "{:.2%}".format(x))
    return df

yearly_rates = (yearly_price_close - yearly_price_open)/yearly_price_open
yearly_rates_p = percent(yearly_rates.copy())
yearly_rates_p.index = yearly_price_open.index.year
yearly_rates_p.tail(15)

In [ ]:
percent(yearly_rates.describe().drop('count'))

In [ ]:
#Exploring Intercorrelations and Distribution of weekly prices 
# print(price)
weekly_price = price.asfreq('W', method='ffill')
# Ommitting Nan values and fetching the weekly value of last day
# print(weekly_price)
weekly_rates = ((weekly_price - weekly_price.shift(1))/weekly_price.shift(1)).iloc[1::,:]
#Obtaining the weekly rates 

In [ ]:
sns.heatmap(weekly_price.corr(), annot=True,linecolor="red",fmt= '.1f')

Almost perfect linear correlation is present between S&P500 and DJIA, but gold shows low values.

In [ ]:
sns.pairplot(weekly_rates)

We can observe a same situation in the Covariance matrix.

In [ ]:
# Before getting into the modeling part, let us check stationarity for both prices and rates of the corresponding assets.
weekly_price.plot(subplots=True, layout=(4,2),figsize=(10,10))

In [ ]:
weekly_rates.plot(subplots=True, layout=(4,2), figsize=(10,10))

It is obvious from the graph that prices show the presence of the upward trend, while rates indicate high possibility for stationarity. Since for the modeling part we need stationary data, I'll test the stationarity of the first difference data.

In [ ]:
#Performing AD Fuller Test
def Stationarity(data, alpha=0.05):
    adf = pd.DataFrame(columns = ['test','p'])
    for col in data.columns:
        df = adfuller(data[col], autolag='AIC')
        adf = pd.concat([adf, pd.DataFrame(np.array(df[0:2]).reshape(1,2),columns = ['test','p'])],axis=0)
    adf['Stationary'] = adf['p'] < alpha 
    # significance level
    adf.index = data.columns
    return adf
# test statistic is lower than the critical value shown, you reject the null hypothesis and 
# infer that the time series is stationary.    
weekly_price_d = weekly_price.diff().dropna()

Stationarity(weekly_price_d)


# ARIMA MODEL

As the procedure is the same with all our variables, I've decided to showcase the steps only for gold and the analog approach can be transferred to S&P500 and DJIA.

## Order Selection

The first step is selecting the appropriate order of the ARIMA model; more accurately, we need to find how many past values (p) and moving averages (q) should be included in the model. Please note that I'm using the first difference of the dataset, so basically, the d parameter is equal to 1.

In [ ]:
fig, axs = plt.subplots(1,2, figsize =(15,6))


plot_acf(weekly_price_d['Gold'], lags=10, ax=axs[0], title='Gold'+': Autocorrelation')
plt.close(2)
plot_pacf(weekly_price_d['Gold'], lags=10, ax=axs[1], title='Gold'+': Partial Autocorrelation')
plt.close(2)

Once we determine the nature of the auto-correlations we use the following rules of thumb.

Rule 1: If the ACF shows exponential decay, the PACF has a spike at lag 1, and no correlation for other lags, then use one autoregressive (p)parameter

Rule 2: If the ACF shows a sine-wave shape pattern or a set of exponential decays, the PACF has spikes at lags 1 and 2, and no correlation for other lags, the use two autoregressive (p) parameters

Rule 3: If the ACF has a spike at lag 1, no correlation for other lags, and the PACF damps out exponentially, then use one moving average (q) parameter.

Rule 4: If the ACF has spikes at lags 1 and 2, no correlation for other lags, and the PACF has a sine-wave shape pattern or a set of exponential decays, then use two moving average (q) parameter.

Rule 5: If the ACF shows exponential decay starting at lag 1, and the PACF shows exponential decay starting at lag 1, then use one autoregressive (p) and one moving average (q) parameter

For order selection, ACF and PACF graphs were used and the value for both p and q is two, which means I'll use ARIMA(2,1,2) model for the actual dataset or ARIMA(2,0,2) for the first difference.

In [ ]:
test_size = 40
train_arima, test_arima = weekly_price_d['Gold'][:-test_size], weekly_price_d['Gold'][-test_size:]
model_arima = ARIMA(train_arima, order=(2,0,2)) 
# d component is 0, so this would actually call ARMA model
model_arima_fitted = model_arima.fit()
model_arima_fitted.summary()

# Forecasting:

In [ ]:
forecast_arima = model_arima_fitted.forecast(steps=test_size)[0]
df_forecast_arima = pd.DataFrame(forecast_arima, index=weekly_price_d.index[-test_size:], columns=['Gold'] )

print('Error:',np.sqrt(mean_squared_error(test_arima, df_forecast_arima)))

In [ ]:
#Lets lower the error by scaling 

error = []
for scaler in (MinMaxScaler(), StandardScaler()): # will use stnadard scaler and minMax sclaer
    train_arima_scaled = scaler.fit_transform(train_arima.values.reshape(-1,1))

    model_arima_scaled = ARIMA(train_arima_scaled, order=(2,0,2))
    model_arima_scaled_fitted = model_arima_scaled.fit()
    forecast_arima_scaled = model_arima_scaled_fitted.forecast(steps=test_size)[0]
    forecast_arima_reverse = scaler.inverse_transform(forecast_arima_scaled.reshape(-1,1))

    df_forecast_arima = pd.DataFrame(forecast_arima_reverse, index=weekly_price_d.index[-test_size:], columns=['Gold'])
    error.append(np.sqrt(mean_squared_error(test_arima, df_forecast_arima)))

pd.DataFrame(error, index = ['MinMaxScaler', 'StandardScaler'], columns = ['Error'])

Error is almost the same so there is no point of scaling the data.

In [ ]:
def inverse_diff(df, df_forecast):
    last_row = df.iloc[-1,:]
    l = []
    num_forecast = df_forecast.shape[0]
    for row in range(num_forecast):
        arr = np.array(last_row + df_forecast.iloc[row,:])
        l.append(arr)
        last_row = arr
#     print(df.index[-1])
#     print(df.index[-1]+timedelta(days=2))
    time = pd.date_range((df.index[-1]+timedelta(days=2)).date(), num_forecast, freq='W')
#     print(df_forecast.index)
#     print(df_forecast.index + timedelta(days = 28))
    result = pd.DataFrame(data=l, columns=df.columns, index=(df_forecast.index+timedelta(days = 28)))
    return result

In [ ]:
model_arima = ARIMA(weekly_price_d['Gold'], order=(2,0,2)) 
model_arima_fitted = model_arima.fit()
prediction_size = 4 # next month
forecast_arima = model_arima_fitted.forecast(steps=prediction_size)[0]
# print(weekly_price_d.index[-prediction_size:])
df_forecast_arima = pd.DataFrame(forecast_arima, index=weekly_price_d.index[-prediction_size:], columns=['Gold'] )
# print("DF Forecast ARIMA:",df_forecast_arima)
weekly_price_forecast_arima = inverse_diff(pd.DataFrame(weekly_price['Gold']), df_forecast_arima)
# weekly_price_forecast_arima.head()

In [ ]:
# print(df_forecast_arima)
weekly_price_forecast_arima = inverse_diff(pd.DataFrame(weekly_price['Gold']), df_forecast_arima)
weekly_price_forecast_arima.head()